<a href="https://colab.research.google.com/github/ISMRM-MIT-CMR/CMR-DL-challenge/blob/master/tutorial_denoising_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Example 01: Denoising on real-valued data**
We first define the data pipelines to feed the data into training, validation and test set. The MNIST database is used for showcasing. A white Gaussian noise is simulated retrospectively and added to the data. The task of the network is to denoise the images with real-valued operations.

To enable GPU support in Google Colab, please go to `Edit -> Notebook settings` and select `GPU` as hardware accelerator.

In [ ]:
# inspect the available GPU hardware
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!nvidia-smi

In [ ]:
# ONLY execute if called from Google Colab or if files are not yet present
! wget https://www.dropbox.com/sh/sag71gy9byxplc3/AACmXeOhXvW8HuLujPaqUPP_a?dl=1 -O dlmri_tutorial.zip
! unzip dlmri_tutorial.zip -d dlmri_tutorial

## Database pipeline
Here starts the main part of the script. First define the data pipelines (in the form of generator functions) for training, validation and test set. Retrospective noise simulation is performed inside the generator functions.

In [ ]:
import tensorflow as tf
import numpy as np
import os
import datetime
import dlmri_tutorial

from dlmri_tutorial.data import DataGeneratorMNIST  # real-valued data generator

# initialize some parameters
noise_level = 0.5  # simulated additive white Gaussian noise level

# Data Generators (Data pipeline) for real-valued data
# training set
training_generator = DataGeneratorMNIST(batch_size=32, 
                                    noise_level=noise_level,
                                    shuffle=True,
                                    mode='train')

# validation set
validation_generator = DataGeneratorMNIST(batch_size=32, 
                                    noise_level=noise_level,
                                    shuffle=False,
                                    mode='val')

# test set
# ideally testing should be performed on real noisy cases and not simulated ones
test_generator = DataGeneratorMNIST(batch_size=1,
                                    noise_level=noise_level,
                                    shuffle=False,
                                    mode='test')

print('Training batches to process:', len(training_generator))
print('Validation batches to process:', len(validation_generator))
print('Test samples to process:', len(test_generator))

## Model
Define the CNN model with its corresponding inputs and outputs.


### 3-layer convolutional neural network (CNN)

In [ ]:
# Generate Model
# Let's start with a 3-layer CNN
input = tf.keras.Input(shape=(28, 28, 1),)  # define input layer and its shape
activation = 'relu'  # select activation function: real-valued ReLU
# convolutional layer 1: Real-valued convolution
conv_out1 = tf.keras.layers.Conv2D(filters=4,                   # output channels, N_fout
                                kernel_size=(3,3),              # kernel size along x and y
                                strides=(1,1),                  # stride performed along x and y
                                padding='SAME',                 # padding of input to adjust output size
                                use_bias=True,                  # learn bias values for conv layer
                                activation=activation)(input)   # apply activation function after conv operation
# convolutional layer 2: Real-valued convolution
conv_out2 = tf.keras.layers.Conv2D(filters=4,
                                kernel_size=(3,3),
                                strides=(1,1),
                                padding='SAME',
                                use_bias=True,
                                activation=activation)(conv_out1)
# convolutional layer 3: Real-valued convolution
output    = tf.keras.layers.Conv2D(filters=1,
                                kernel_size=(3,3),
                                strides=(1,1),
                                padding='SAME',
                                use_bias=True,
                                activation=activation)(conv_out2)

# instantiate a keras functional model: combine layers into a model with specified inputs and outputs
model = tf.keras.Model(input, output, name='3layerCNN')

# print model overview
model.summary()

### 3-layer residual convolutional neural network (CNN)

In [ ]:
# Generate Model
# Let's start with a residual 3-layer CNN
input = tf.keras.Input(shape=(28, 28, 1),)  # define input layer and its shape
activation = 'relu'  # select activation function
# convolutional layer 1
conv_out1 = tf.keras.layers.Conv2D(filters=4,                   # output channels, N_fout
                                kernel_size=(3,3),              # kernel size along x and y
                                strides=(1,1),                  # stride performed along x and y
                                padding='SAME',                 # padding of input to adjust output size
                                use_bias=True,                  # learn bias values for conv layer
                                activation=activation)(input)   # apply activation function after conv operation
# convolutional layer 2
conv_out2 = tf.keras.layers.Conv2D(filters=4,
                                kernel_size=(3,3),
                                strides=(1,1),
                                padding='SAME',
                                use_bias=True,
                                activation=activation)(conv_out1)
# convolutional layer 3
residual    = tf.keras.layers.Conv2D(filters=1,
                                kernel_size=(3,3),
                                strides=(1,1),
                                padding='SAME',
                                use_bias=True,
                                activation=activation)(conv_out2)
# residual connection
output = tf.keras.layers.Add()([input, residual])

# instantiate a keras functional model: combine layers into a model with specified inputs and outputs
model = tf.keras.Model(input, output, name='Residual3layerCNN')

# print model overview
model.summary()

### Build model
Compile the model, assign an optimizer, loss function and validation metrics. Prepare some keras callbacks to monitor training progress.

In [ ]:
# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),      # used optimizer with chosen learning rate
              loss='mse',                                                   # loss function 
              metrics=['mse', 'mae'])                                       # evaluation metrics (for training and validation set)

# define callbacks to monitor model
keras_callbacks = dlmri_tutorial.get_callbacks(validation_generator, model)    

## Tensorboard
Start the Tensorboard [optional] to monitor training progress and display validation outputs.

In [ ]:
# start Tensorboard
%load_ext tensorboard
%tensorboard --logdir=logs

## Training
Train the configured and compiled model. Monitor training progress with validation set.

In [ ]:
# train model with training set and evaluate its performance with the validation set
model.fit(training_generator,                     # training set
          validation_data=validation_generator,   # validation set
          epochs=3,                               # number of epochs to train the model
          callbacks=keras_callbacks)              # callbacks to monitor or control training


## Testing
Test the trained model to predict a denoised output and to display performance (metrics) on test set.

In [ ]:
# predict with trained model
predicted_output = model.predict(test_generator)

# evaluate trained model
loss_metric_test = model.evaluate(test_generator)

# display the predicted output
import matplotlib.pyplot as plt
icase = 0  # display the first example
plt.imshow(np.squeeze(predicted_output[icase,]), cmap=plt.gray())
plt.show()